#### CODE
SBERT(x1='Aku Sedih', x2='Aku Senang') # Mono
SBERT(x1='Aku Sedih', x2='I am Happy') # Cross

#### Mono ICL
Predict the sentiment of the the following sentences:
Aku senang => positive
Aku Bangga => positive
Aku Sedih => ???

#### Cross ICL
Predict the sentiment of the the following sentences:
I am happy => positive
You are sad => pegative
Aku Sedih => ???

#### Mono / Cross IIA
Predict the sentiment of the the following sentences:
I am happy => Aku Senang
You are sad => Kamu Sedih
Aku Sedih => ???

#### Cross ICL + IIA
Predict the sentiment of the the following sentences:
I am happy=> positive
You are sad => pegative

I am happy => Aku Senang
You are sad => Kamu Sedih

Aku Sedih => ???

#### Cross ICL + IIA + IOA
Predict the sentiment of the the following sentences:
I am happy => positive
You are sad => pegative
You are sad => pegative

I am happy => Aku Senang
You are sad => Kamu Sedih
You are sad => Kamu Sedih

In Indonesian, positive means positif and negative means negatif

Aku Sedih => ???

#### ITC
Predict the sentiment of the the following sentences:
| Ind Text | Ind Label | Eng Label | Eng Text |
------------------------------------------------
| Aku Senang |  Negative | Negative | I am Happy |
| You are sad |  Positive | Positive | I am Sad |
| You are sad |  Positive | Positive | I am Sad |
| Aku Sedih |  ?????? | |

In [34]:
import pandas as pd
import numpy as np
import glob
from IPython.display import display, HTML

# ICL

In [18]:
dfs = []
for path in glob.glob('../metrics/*.csv'):
    meta = path.split('/')[-1][:-4].split('_')
    dset_name, model_name = meta[1], meta[2]
    
    icl_meta = meta[3][4:]
    num_icl = int(icl_meta[-1])
    icl_type = icl_meta.split('-')[0]
    icl_idx = icl_meta[len(icl_type)+1:-2]
        
    iia_meta = meta[4][4:]
    num_iia = int(iia_meta[-1])
    iia_type = iia_meta.split('-')[0]
    iia_idx = iia_meta[len(iia_type)+1:-2]
    
    use_ioa = meta[5].split('-')[1]
    
    # print(f'{dset_name} {model_name} {use_ioa}')
    # print(f'{icl_type} {num_icl} {icl_idx}')
    # print(f'{iia_type} {num_iia} {iia_idx}')
    
    # Load Metrics
    df = pd.read_csv(path)
    df = df.T
    df.columns = df.head(1).values[0]
    df = df.iloc[1:]

    # Inject Metadata
    df['model_name'] = model_name
    df['num_icl'] = num_icl
    df['icl_type'] = icl_type
    df['icl_idx'] = icl_idx
    df['num_iia'] = num_iia
    df['iia_type'] = iia_type
    df['iia_idx'] = iia_idx
    df['use_ioa'] = use_ioa
    
    df['accuracy'] = pd.to_numeric(df['accuracy'])
    df['macro_f1'] = pd.to_numeric(df['macro_f1'])
    df['weighted_f1'] = pd.to_numeric(df['weighted_f1'])
    
    # Change Dataset Experiments
    df['dataset'] = df['dataset'].apply(lambda x: 'tweetsentimulti' if x == 'tweet_senti_multi' else x)
    
    # Remove Unused Experiments
    df = df.loc[(df['icl_type'] != 'cross') | (~df['icl_idx'].isin(['tf-idf', 'count'])),:]
    df = df.loc[(df['iia_type'] != 'cross') | (~df['iia_idx'].isin(['tf-idf', 'count'])),:]
    df = df.loc[(df['lang'] != 'eng'),:]
    dfs.append(df)
    
df = pd.concat(dfs)

In [5]:
df.to_csv('icl_results.csv', index=False)

In [33]:
df.loc[(df['icl_type'] == 'none') & (df['use_ioa'] == 'False'),:].groupby(
    ['dataset','model_name']
)[['accuracy', 'macro_f1', 'weighted_f1']].mean()

accuracy  macro_f1  weighted_f1
dataset             model_name                                 
americasnli         bloom-7b1   0.335009  0.183323     0.183294
                    xglm-7.5B   0.335779  0.295794     0.295804
americasnli-spa     bloom-7b1   0.337471  0.238957     0.238927
                    xglm-7.5B   0.338525  0.270561     0.270580
masakhanews         bloom-7b1   0.541977  0.499331     0.531356
                    xglm-7.5B   0.455480  0.411192     0.438588
nusatranslation     bloom-7b1   0.691545  0.613276     0.682906
                    xglm-7.5B   0.586094  0.560478     0.600564
nusatranslation-ind bloom-7b1   0.687813  0.558422     0.653980
                    xglm-7.5B   0.650608  0.583103     0.651173
tweetsentimulti     bloom-7b1   0.448378  0.428912     0.428912
                    xglm-7.5B   0.448009  0.406417     0.406417

In [8]:
df.groupby(['model_name', 'dataset']).size()

model_name  dataset            
bloom-7b1   americasnli            296
            americasnli-spa        296
            masakhanews            396
            nusatranslation        264
            nusatranslation-ind    264
            tweetsentimulti        308
xglm-7.5B   americasnli            296
            americasnli-spa        296
            masakhanews            396
            nusatranslation        264
            nusatranslation-ind    264
            tweetsentimulti        308
dtype: int64

In [9]:
df.groupby(['dataset', 'lang']).size()

dataset              lang
americasnli          aym     88
                     bzd     40
                     cni     40
                     gn      88
                     hch     40
                     nah     40
                     oto     88
                     quy     88
                     shp     40
                     tar     40
americasnli-spa      aym     88
                     bzd     40
                     cni     40
                     gn      88
                     hch     40
                     nah     40
                     oto     88
                     quy     88
                     shp     40
                     tar     40
masakhanews          amh     88
                     hau     88
                     ibo     88
                     lug     88
                     pcm     88
                     sna     88
                     swa     88
                     xho     88
                     yor     88
nusatranslation      btk     88
              

In [10]:
for dset in df['dataset'].unique():
    print(dset)
    display(df.loc[
        (df['dataset'] == dset) & (df['model_name'] == 'bloom-7b1'),:
    ].sort_values(['dataset', 'icl_type', 'iia_type', 'use_ioa'], ascending=False).reset_index(drop=True).groupby(
        ['dataset', 'icl_type', 'iia_type', 'icl_idx', 'iia_idx', 'num_icl', 'num_iia', 'use_ioa']
    )[['accuracy', 'macro_f1', 'weighted_f1']].mean())

masakhanews


accuracy  \
dataset     icl_type iia_type icl_idx            iia_idx            num_icl num_iia use_ioa             
masakhanews cross    cross    sbert              sbert              3       3       False    0.561995   
                                                                                    Target   0.378480   
                                                                                    True     0.271840   
                     mono     sbert              sbert              3       3       False    0.570330   
                                                                                    Target   0.375364   
                                                                                    True     0.297698   
                     none     count$tf-idf$sbert random             3       0       False    0.571759   
                              sbert              random             3       0       False    0.579809   
                                                                                    Target   0.396602   
                                                                                    True     0.307508   
            mono     cross    sbert              sbert              3       3       False    0.686563   
                                                                                    Target   0.598273   
                                                                                    True     0.302950   
                     mono     random             random             3       3       False    0.566352   
                                                                                    Target   0.333483   
                                                                                    True     0.290122   
                              sbert              sbert              3       3       False    0.698081   
                                                                                    Target   0.582857   
                                                                                    True     0.299425   
                     none     count              random             3       0       False    0.675234   
                              count$tf-idf$sbert random             3       0       False    0.700138   
                              random             random             3       0       False    0.572081   
                                                                                    Target   0.359786   
                                                                                    True     0.264934   
                              sbert              random             3       0       False    0.674767   
                                                                                    Target   0.620948   
                                                                                    True     0.272009   
                              tf-idf             random             3       0       False    0.684143   
            none     cross    random             count$tf-idf$sbert 0       3       False    0.539126   
                                                 sbert              0       3       False    0.519455   
                                                                                    Target   0.284209   
                                                                                    True     0.264681   
                     mono     random             count              0       3       False    0.540033   
                                                 count$tf-idf$sbert 0       3       False    0.559851   
                                                 random             0       3       False    0.542531   
                                                                                    Target   0.276160   
                                                                                    True     0.263815  

americasnli


accuracy  \
dataset     icl_type iia_type icl_idx            iia_idx            num_icl num_iia use_ioa             
americasnli cross    cross    sbert              sbert              3       3       False    0.339031   
                                                                                    Target   0.342888   
                                                                                    True     0.331221   
                     mono     sbert              sbert              3       3       False    0.351830   
                                                                                    Target   0.346227   
                                                                                    True     0.332557   
                     none     count$tf-idf$sbert random             3       0       False    0.342347   
                              sbert              random             3       0       False    0.340602   
                                                                                    Target   0.343219   
                                                                                    True     0.331888   
            mono     cross    sbert              sbert              3       3       False    0.348615   
                                                                                    Target   0.344558   
                                                                                    True     0.335889   
                     mono     random             random             3       3       False    0.331451   
                                                                                    Target   0.320225   
                                                                                    True     0.332890   
                              sbert              sbert              3       3       False    0.346246   
                                                                                    Target   0.355555   
                                                                                    True     0.340556   
                     none     count              random             3       0       False    0.356281   
                              count$tf-idf$sbert random             3       0       False    0.344783   
                              random             random             3       0       False    0.329317   
                                                                                    Target   0.339872   
                                                                                    True     0.334224   
                              sbert              random             3       0       False    0.341287   
                                                                                    Target   0.356224   
                                                                                    True     0.336557   
                              tf-idf             random             3       0       False    0.348530   
            none     cross    random             count$tf-idf$sbert 0       3       False    0.335163   
                                                 sbert              0       3       False    0.335822   
                                                                                    Target   0.332556   
                                                                                    True     0.334560   
                     mono     random             count              0       3       False    0.335295   
                                                 count$tf-idf$sbert 0       3       False    0.334228   
                                                 random             0       3       False    0.333559   
                                                                                    Target   0.318554   
                                                                                    True     0.329217  

americasnli-spa


...

accuracy  \
dataset         icl_type iia_type icl_idx            iia_idx            num_icl num_iia use_ioa             
nusatranslation cross    cross    sbert              sbert              3       3       False    0.702222   
                                                                                        Target   0.688889   
                                                                                        True     0.449861   
                         mono     sbert              sbert              3       3       False    0.715417   
                                                                                        Target   0.684444   
                                                                                        True     0.670833   
                         none     count$tf-idf$sbert random             3       0       False    0.705139   
                                  sbert              random             3       0       False    0.735139   
                                                                                        Target   0.649444   
                                                                                        True     0.494444   
                mono     cross    sbert              sbert              3       3       False    0.721111   
                                                                                        Target   0.631944   
                                                                                        True     0.705000   
                         mono     random             random             3       3       False    0.641250   
                                                                                        Target   0.701250   
                                                                                        True     0.704028   
                                  sbert              sbert              3       3       False    0.736389   
                                                                                        Target   0.631111   
                                                                                        True     0.706806   
                         none     count              random             3       0       False    0.693750   
                                  count$tf-idf$sbert random             3       0       False    0.718333   
                                  random             random             3       0       False    0.719028   
                                                                                        Target   0.643056   
                                                                                        True     0.715972   
                                  sbert              random             3       0       False    0.682500   
                                                                                        Target   0.678333   
                                                                                        True     0.709167   
                                  tf-idf             random             3       0       False    0.699722   
                none     cross    random             count$tf-idf$sbert 0       3       False    0.708889   
                                                     sbert              0       3       False    0.662639   
                                                                                        Target   0.696250   
                                                                                        True     0.704722   
                         mono     random             count              0       3       False    0.708750   
                                                     count$tf-idf$sbert 0       3       False    0.714028   
                                                     random             0       3       False    0.664028   
                                                                

tweetsentimulti


accuracy  \
dataset         icl_type iia_type icl_idx            iia_idx            num_icl num_iia use_ioa             
tweetsentimulti cross    cross    sbert              sbert              3       3       False    0.505090   
                                                                                        Target   0.460427   
                                                                                        True     0.418555   
                         mono     sbert              sbert              3       3       False    0.493924   
                                                                                        Target   0.464696   
                                                                                        True     0.361576   
                         none     count$tf-idf$sbert random             3       0       False    0.544828   
                                  sbert              random             3       0       False    0.533333   
                                                                                        Target   0.508539   
                                                                                        True     0.382266   
                mono     cross    sbert              sbert              3       3       False    0.521346   
                                                                                        Target   0.498030   
                                                                                        True     0.409195   
                         mono     random             random             3       3       False    0.454680   
                                                                                        Target   0.420033   
                                                                                        True     0.408210   
                                  sbert              sbert              3       3       False    0.525287   
                                                                                        Target   0.505090   
                                                                                        True     0.371100   
                         none     count              random             3       0       False    0.492775   
                                  count$tf-idf$sbert random             3       0       False    0.533990   
                                  random             random             3       0       False    0.455665   
                                                                                        Target   0.419376   
                                                                                        True     0.362726   
                                  sbert              random             3       0       False    0.532841   
                                                                                        Target   0.519048   
                                                                                        True     0.352217   
                                  tf-idf             random             3       0       False    0.497701   
                none     cross    random             count$tf-idf$sbert 0       3       False    0.427750   
                                                     sbert              0       3       False    0.427094   
                                                                                        Target   0.375369   
                                                                                        True     0.382102   
                         mono     random             count              0       3       False    0.448933   
                                                     count$tf-idf$sbert 0       3       False    0.446634   
                                                     random             0       3       False    0.449261   
                                                                

nusatranslation-ind


accuracy  \
dataset             icl_type iia_type icl_idx            iia_idx            num_icl num_iia use_ioa             
nusatranslation-ind cross    cross    sbert              sbert              3       3       False    0.700139   
                                                                                            Target   0.612639   
                                                                                            True     0.647778   
                             mono     sbert              sbert              3       3       False    0.655000   
                                                                                            Target   0.656667   
                                                                                            True     0.669861   
                             none     count$tf-idf$sbert random             3       0       False    0.734722   
                                      sbert              random             3       0       False    0.735694   
                                                                                            Target   0.728333   
                                                                                            True     0.707222   
                    mono     cross    sbert              sbert              3       3       False    0.633750   
                                                                                            Target   0.655417   
                                                                                            True     0.359583   
                             mono     random             random             3       3       False    0.621667   
                                                                                            Target   0.452639   
                                                                                            True     0.342500   
                                      sbert              sbert              3       3       False    0.685972   
                                                                                            Target   0.663472   
                                                                                            True     0.367500   
                             none     count              random             3       0       False    0.707639   
                                      count$tf-idf$sbert random             3       0       False    0.694583   
                                      random             random             3       0       False    0.723056   
                                                                                            Target   0.721806   
                                                                                            True     0.536944   
                                      sbert              random             3       0       False    0.697222   
                                                                                            Target   0.720139   
                                                                                            True     0.712500   
                                      tf-idf             random             3       0       False    0.708056   
                    none     cross    random             count$tf-idf$sbert 0       3       False    0.643333   
                                                         sbert              0       3       False    0.645694   
                                                                                            Target   0.513750   
                                                                                            True     0.395556   
                             mono     random             count              0       3       False    0.699028   
                                                         count$tf-idf$sbert 0       3       False    0.708611   
                                 

In [12]:
for dset in df['dataset'].unique():
    print(dset)
    display(df.loc[
        (df['dataset'] == dset) & (df['model_name'] == 'xglm-7.5B'),:
    ].sort_values(['dataset', 'icl_type', 'iia_type', 'use_ioa'], ascending=False).reset_index(drop=True).groupby(
        ['dataset', 'icl_type', 'iia_type', 'icl_idx', 'iia_idx', 'num_icl', 'num_iia', 'use_ioa']
    )[['accuracy', 'macro_f1', 'weighted_f1']].mean())

masakhanews


accuracy  \
dataset     icl_type iia_type icl_idx            iia_idx            num_icl num_iia use_ioa             
masakhanews cross    cross    sbert              sbert              3       3       False    0.543885   
                                                                                    Target   0.364381   
                                                                                    True     0.318981   
                     mono     sbert              sbert              3       3       False    0.554672   
                                                                                    Target   0.363117   
                                                                                    True     0.318347   
                     none     count$tf-idf$sbert random             3       0       False    0.582451   
                              sbert              random             3       0       False    0.582991   
                                                                                    Target   0.391624   
                                                                                    True     0.331273   
            mono     cross    sbert              sbert              3       3       False    0.664797   
                                                                                    Target   0.571977   
                                                                                    True     0.344484   
                     mono     random             random             3       3       False    0.510667   
                                                                                    Target   0.343354   
                                                                                    True     0.308260   
                              sbert              sbert              3       3       False    0.662971   
                                                                                    Target   0.564102   
                                                                                    True     0.330966   
                     none     count              random             3       0       False    0.645179   
                              count$tf-idf$sbert random             3       0       False    0.668623   
                              random             random             3       0       False    0.528734   
                                                                                    Target   0.353483   
                                                                                    True     0.309874   
                              sbert              random             3       0       False    0.648209   
                                                                                    Target   0.606429   
                                                                                    True     0.344137   
                              tf-idf             random             3       0       False    0.655084   
            none     cross    random             count$tf-idf$sbert 0       3       False    0.447890   
                                                 sbert              0       3       False    0.461811   
                                                                                    Target   0.273523   
                                                                                    True     0.274370   
                     mono     random             count              0       3       False    0.473780   
                                                 count$tf-idf$sbert 0       3       False    0.444569   
                                                 random             0       3       False    0.428342   
                                                                                    Target   0.275807   
                                                                                    True     0.271467  

americasnli


accuracy  \
dataset     icl_type iia_type icl_idx            iia_idx            num_icl num_iia use_ioa             
americasnli cross    cross    sbert              sbert              3       3       False    0.334891   
                                                                                    Target   0.345889   
                                                                                    True     0.333884   
                     mono     sbert              sbert              3       3       False    0.346319   
                                                                                    Target   0.351239   
                                                                                    True     0.330545   
                     none     count$tf-idf$sbert random             3       0       False    0.342489   
                              sbert              random             3       0       False    0.343968   
                                                                                    Target   0.339884   
                                                                                    True     0.331549   
            mono     cross    sbert              sbert              3       3       False    0.352144   
                                                                                    Target   0.367900   
                                                                                    True     0.327884   
                     mono     random             random             3       3       False    0.333154   
                                                                                    Target   0.345905   
                                                                                    True     0.333561   
                              sbert              sbert              3       3       False    0.344259   
                                                                                    Target   0.352885   
                                                                                    True     0.341900   
                     none     count              random             3       0       False    0.358276   
                              count$tf-idf$sbert random             3       0       False    0.344129   
                              random             random             3       0       False    0.333581   
                                                                                    Target   0.331873   
                                                                                    True     0.337558   
                              sbert              random             3       0       False    0.348803   
                                                                                    Target   0.363556   
                                                                                    True     0.338556   
                              tf-idf             random             3       0       False    0.345837   
            none     cross    random             count$tf-idf$sbert 0       3       False    0.334917   
                                                 sbert              0       3       False    0.338113   
                                                                                    Target   0.338555   
                                                                                    True     0.342892   
                     mono     random             count              0       3       False    0.334662   
                                                 count$tf-idf$sbert 0       3       False    0.331409   
                                                 random             0       3       False    0.339574   
                                                                                    Target   0.346221   
                                                                                    True     0.336230  

americasnli-spa


...

accuracy  \
dataset         icl_type iia_type icl_idx            iia_idx            num_icl num_iia use_ioa             
nusatranslation cross    cross    sbert              sbert              3       3       False    0.540972   
                                                                                        Target   0.610000   
                                                                                        True     0.655833   
                         mono     sbert              sbert              3       3       False    0.553333   
                                                                                        Target   0.636389   
                                                                                        True     0.654861   
                         none     count$tf-idf$sbert random             3       0       False    0.574861   
                                  sbert              random             3       0       False    0.617917   
                                                                                        Target   0.557222   
                                                                                        True     0.564722   
                mono     cross    sbert              sbert              3       3       False    0.648611   
                                                                                        Target   0.628611   
                                                                                        True     0.519167   
                         mono     random             random             3       3       False    0.525694   
                                                                                        Target   0.502917   
                                                                                        True     0.640972   
                                  sbert              sbert              3       3       False    0.601111   
                                                                                        Target   0.638889   
                                                                                        True     0.514028   
                         none     count              random             3       0       False    0.618194   
                                  count$tf-idf$sbert random             3       0       False    0.608056   
                                  random             random             3       0       False    0.600972   
                                                                                        Target   0.550694   
                                                                                        True     0.502361   
                                  sbert              random             3       0       False    0.622361   
                                                                                        Target   0.485833   
                                                                                        True     0.427639   
                                  tf-idf             random             3       0       False    0.554444   
                none     cross    random             count$tf-idf$sbert 0       3       False    0.535833   
                                                     sbert              0       3       False    0.610833   
                                                                                        Target   0.536806   
                                                                                        True     0.559583   
                         mono     random             count              0       3       False    0.576667   
                                                     count$tf-idf$sbert 0       3       False    0.628889   
                                                     random             0       3       False    0.622917   
                                                                

tweetsentimulti


accuracy  \
dataset         icl_type iia_type icl_idx            iia_idx            num_icl num_iia use_ioa             
tweetsentimulti cross    cross    sbert              sbert              3       3       False    0.464532   
                                                                                        Target   0.478325   
                                                                                        True     0.414122   
                         mono     sbert              sbert              3       3       False    0.478325   
                                                                                        Target   0.475862   
                                                                                        True     0.411987   
                         none     count$tf-idf$sbert random             3       0       False    0.466174   
                                  sbert              random             3       0       False    0.458292   
                                                                                        Target   0.443350   
                                                                                        True     0.390476   
                mono     cross    sbert              sbert              3       3       False    0.495567   
                                                                                        Target   0.500985   
                                                                                        True     0.434319   
                         mono     random             random             3       3       False    0.441215   
                                                                                        Target   0.415435   
                                                                                        True     0.402791   
                                  sbert              sbert              3       3       False    0.496880   
                                                                                        Target   0.494417   
                                                                                        True     0.421675   
                         none     count              random             3       0       False    0.442200   
                                  count$tf-idf$sbert random             3       0       False    0.471100   
                                  random             random             3       0       False    0.388342   
                                                                                        Target   0.391461   
                                                                                        True     0.359442   
                                  sbert              random             3       0       False    0.469294   
                                                                                        Target   0.465353   
                                                                                        True     0.384893   
                                  tf-idf             random             3       0       False    0.433990   
                none     cross    random             count$tf-idf$sbert 0       3       False    0.452053   
                                                     sbert              0       3       False    0.444499   
                                                                                        Target   0.424302   
                                                                                        True     0.410181   
                         mono     random             count              0       3       False    0.442036   
                                                     count$tf-idf$sbert 0       3       False    0.453366   
                                                     random             0       3       False    0.454187   
                                                                

nusatranslation-ind


accuracy  \
dataset             icl_type iia_type icl_idx            iia_idx            num_icl num_iia use_ioa             
nusatranslation-ind cross    cross    sbert              sbert              3       3       False    0.595972   
                                                                                            Target   0.536250   
                                                                                            True     0.608472   
                             mono     sbert              sbert              3       3       False    0.666667   
                                                                                            Target   0.591111   
                                                                                            True     0.483472   
                             none     count$tf-idf$sbert random             3       0       False    0.624583   
                                      sbert              random             3       0       False    0.625139   
                                                                                            Target   0.520556   
                                                                                            True     0.468889   
                    mono     cross    sbert              sbert              3       3       False    0.670139   
                                                                                            Target   0.617361   
                                                                                            True     0.640139   
                             mono     random             random             3       3       False    0.653611   
                                                                                            Target   0.456111   
                                                                                            True     0.440000   
                                      sbert              sbert              3       3       False    0.669167   
                                                                                            Target   0.600139   
                                                                                            True     0.470833   
                             none     count              random             3       0       False    0.568056   
                                      count$tf-idf$sbert random             3       0       False    0.608889   
                                      random             random             3       0       False    0.594861   
                                                                                            Target   0.556250   
                                                                                            True     0.501806   
                                      sbert              random             3       0       False    0.547222   
                                                                                            Target   0.570000   
                                                                                            True     0.502639   
                                      tf-idf             random             3       0       False    0.562639   
                    none     cross    random             count$tf-idf$sbert 0       3       False    0.659028   
                                                         sbert              0       3       False    0.654583   
                                                                                            Target   0.581250   
                                                                                            True     0.537361   
                             mono     random             count              0       3       False    0.653889   
                                                         count$tf-idf$sbert 0       3       False    0.658750   
                                 

### Masakhanews

In [6]:
df.loc[
    (df['icl_idx'] == 'sbert') & (df['num_icl'] == 3) & 
    (df['dataset'] == 'masakhanews') & (df['use_ioa'] == 'False')
,:].sort_values(['dataset', 'icl_type', 'iia_type'], ascending=False).reset_index(drop=True).groupby(
    ['dataset', 'lang']
)[['accuracy', 'macro_f1', 'weighted_f1']].mean()

accuracy  macro_f1  weighted_f1
dataset     lang                                 
masakhanews amh   0.574088  0.561374     0.563506
            hau   0.548778  0.530443     0.544731
            ibo   0.743223  0.714396     0.739697
            lug   0.528507  0.427796     0.515521
            pcm   0.909133  0.884584     0.910033
            sna   0.565234  0.553240     0.546229
            swa   0.716987  0.663724     0.722729
            xho   0.354497  0.291473     0.381852
            yor   0.742092  0.746492     0.743316

In [9]:
df.loc[
    (df['icl_idx'] == 'sbert') & (df['num_icl'] == 3) & 
    (df['dataset'] == 'masakhanews') & (df['use_ioa'] == 'True')
,:].sort_values(['dataset', 'icl_type', 'iia_type'], ascending=False).reset_index(drop=True).groupby(
    ['dataset', 'lang']
)[['accuracy', 'macro_f1', 'weighted_f1']].mean()

accuracy  macro_f1  weighted_f1
dataset     lang                                 
masakhanews amh   0.261525  0.205631     0.210955
            hau   0.195709  0.131636     0.136747
            ibo   0.183333  0.135556     0.142224
            lug   0.156951  0.136444     0.102395
            pcm   0.871038  0.840434     0.876496
            sna   0.310298  0.194653     0.196871
            swa   0.132353  0.119898     0.145936
            xho   0.170034  0.103689     0.125318
            yor   0.345904  0.292395     0.302593

In [10]:
df.loc[
    (df['icl_idx'] == 'sbert') & (df['num_icl'] == 3) & 
    (df['dataset'] == 'masakhanews') & (df['use_ioa'] == 'Target')
,:].sort_values(['dataset', 'icl_type', 'iia_type'], ascending=False).reset_index(drop=True).groupby(
    ['dataset', 'lang']
)[['accuracy', 'macro_f1', 'weighted_f1']].mean()

accuracy  macro_f1  weighted_f1
dataset     lang                                 
masakhanews amh   0.602726  0.598121     0.601579
            hau   0.396586  0.374742     0.384499
            ibo   0.453205  0.423071     0.450312
            lug   0.380045  0.291256     0.369467
            pcm   0.906148  0.879261     0.906903
            sna   0.471206  0.468114     0.461676
            swa   0.495011  0.435504     0.503198
            xho   0.250421  0.206158     0.246311
            yor   0.489051  0.475155     0.475712

### AmericasNLI

In [11]:
df.loc[
    (df['icl_idx'] == 'sbert') & (df['num_icl'] == 3) & 
    (df['dataset'] == 'americasnli') & (df['use_ioa'] == 'False')
,:].sort_values(['dataset', 'icl_type', 'iia_type'], ascending=False).reset_index(drop=True).groupby(
    ['dataset', 'lang']
)[['accuracy', 'macro_f1', 'weighted_f1']].mean()

accuracy  macro_f1  weighted_f1
dataset     lang                                 
americasnli aym   0.349500  0.330215     0.330215
            bzd   0.341000  0.323159     0.323159
            cni   0.346167  0.325843     0.325843
            gn    0.362500  0.349273     0.349273
            hch   0.339833  0.319649     0.319649
            nah   0.338584  0.322497     0.322410
            oto   0.335394  0.315070     0.314955
            quy   0.352333  0.335195     0.335195
            shp   0.335167  0.312287     0.312287
            tar   0.350000  0.328639     0.328639

In [12]:
df.loc[
    (df['icl_idx'] == 'sbert') & (df['num_icl'] == 3) & 
    (df['dataset'] == 'americasnli') & (df['use_ioa'] == 'True')
,:].sort_values(['dataset', 'icl_type', 'iia_type'], ascending=False).reset_index(drop=True).groupby(
    ['dataset', 'lang']
)[['accuracy', 'macro_f1', 'weighted_f1']].mean()

accuracy  macro_f1  weighted_f1
dataset     lang                                 
americasnli aym   0.339556  0.236021     0.236021
            gn    0.340000  0.223675     0.223675
            oto   0.333779  0.174341     0.174776
            quy   0.325778  0.216994     0.216994

In [13]:
df.loc[
    (df['icl_idx'] == 'sbert') & (df['num_icl'] == 3) & 
    (df['dataset'] == 'americasnli') & (df['use_ioa'] == 'Target')
,:].sort_values(['dataset', 'icl_type', 'iia_type'], ascending=False).reset_index(drop=True).groupby(
    ['dataset', 'lang']
)[['accuracy', 'macro_f1', 'weighted_f1']].mean()

accuracy  macro_f1  weighted_f1
dataset     lang                                 
americasnli aym   0.352333  0.342849     0.342849
            gn    0.354667  0.332101     0.332101
            oto   0.333389  0.310899     0.310959
            quy   0.355667  0.321994     0.321994

# AmericasNLI Spa

In [14]:
df.loc[
    (df['icl_idx'] == 'sbert') & (df['num_icl'] == 3) & 
    (df['dataset'] == 'americasnli-spa') & (df['use_ioa'] == 'False')
,:].sort_values(['dataset', 'icl_type', 'iia_type'], ascending=False).reset_index(drop=True).groupby(
    ['dataset', 'lang']
)[['accuracy', 'macro_f1', 'weighted_f1']].mean()

accuracy  macro_f1  weighted_f1
dataset         lang                                 
americasnli-spa aym   0.331000  0.303291     0.303291
                bzd   0.331667  0.310663     0.310663
                cni   0.343000  0.319797     0.319797
                gn    0.337000  0.313351     0.313351
                hch   0.335667  0.306416     0.306416
                nah   0.341294  0.311806     0.312037
                oto   0.341578  0.325266     0.325343
                quy   0.343333  0.312039     0.312039
                shp   0.346500  0.325844     0.325844
                tar   0.336333  0.317017     0.317017

In [15]:
df.loc[
    (df['icl_idx'] == 'sbert') & (df['num_icl'] == 3) & 
    (df['dataset'] == 'americasnli-spa') & (df['use_ioa'] == 'True')
,:].sort_values(['dataset', 'icl_type', 'iia_type'], ascending=False).reset_index(drop=True).groupby(
    ['dataset', 'lang']
)[['accuracy', 'macro_f1', 'weighted_f1']].mean()

accuracy  macro_f1  weighted_f1
dataset         lang                                 
americasnli-spa aym   0.335111  0.262810     0.262810
                gn    0.340889  0.261620     0.261620
                oto   0.336453  0.283127     0.283308
                quy   0.334444  0.188789     0.188789

In [16]:
df.loc[
    (df['icl_idx'] == 'sbert') & (df['num_icl'] == 3) & 
    (df['dataset'] == 'americasnli-spa') & (df['use_ioa'] == 'Target')
,:].sort_values(['dataset', 'icl_type', 'iia_type'], ascending=False).reset_index(drop=True).groupby(
    ['dataset', 'lang']
)[['accuracy', 'macro_f1', 'weighted_f1']].mean()

accuracy  macro_f1  weighted_f1
dataset         lang                                 
americasnli-spa aym   0.342833  0.321935     0.321935
                gn    0.347833  0.323214     0.323214
                oto   0.344418  0.323075     0.323156
                quy   0.349333  0.314742     0.314742

### NusaTranslation

In [17]:
df.loc[
    (df['icl_idx'] == 'sbert') & (df['num_icl'] == 3) & 
    (df['dataset'] == 'nusatranslation') & (df['use_ioa'] == 'False')
,:].sort_values(['dataset', 'icl_type', 'iia_type'], ascending=False).reset_index(drop=True).groupby(
    ['dataset', 'lang']
)[['accuracy', 'macro_f1', 'weighted_f1']].mean()

accuracy  macro_f1  weighted_f1
dataset         lang                                 
nusatranslation btk   0.666667  0.560681     0.647696
                jav   0.702604  0.601382     0.683119
                mad   0.669167  0.561765     0.649481
                mak   0.674167  0.550413     0.644875
                min   0.726458  0.629749     0.707227
                sun   0.710729  0.601039     0.685815

In [18]:
df.loc[
    (df['icl_idx'] == 'sbert') & (df['num_icl'] == 3) & 
    (df['dataset'] == 'nusatranslation') & (df['use_ioa'] == 'True')
,:].sort_values(['dataset', 'icl_type', 'iia_type'], ascending=False).reset_index(drop=True).groupby(
    ['dataset', 'lang']
)[['accuracy', 'macro_f1', 'weighted_f1']].mean()

accuracy  macro_f1  weighted_f1
dataset         lang                                 
nusatranslation btk   0.615000  0.464031     0.554566
                jav   0.626667  0.494678     0.576599
                mad   0.580000  0.452134     0.526362
                mak   0.590278  0.407228     0.502220
                min   0.666250  0.536641     0.625500
                sun   0.657917  0.514197     0.607224

In [19]:
df.loc[
    (df['icl_idx'] == 'sbert') & (df['num_icl'] == 3) & 
    (df['dataset'] == 'nusatranslation') & (df['use_ioa'] == 'Target')
,:].sort_values(['dataset', 'icl_type', 'iia_type'], ascending=False).reset_index(drop=True).groupby(
    ['dataset', 'lang']
)[['accuracy', 'macro_f1', 'weighted_f1']].mean()

accuracy  macro_f1  weighted_f1
dataset         lang                                 
nusatranslation btk   0.586875  0.541443     0.592497
                jav   0.653438  0.596888     0.654544
                mad   0.601667  0.552363     0.608014
                mak   0.586042  0.531152     0.587981
                min   0.668750  0.613654     0.670295
                sun   0.658646  0.606605     0.658824

### Tweet Senti Multi

In [20]:
df.loc[
    (df['icl_idx'] == 'sbert') & (df['num_icl'] == 3) & 
    (df['dataset'] == 'tweetsentimulti') & (df['use_ioa'] == 'False')
,:].sort_values(['dataset', 'icl_type', 'iia_type'], ascending=False).reset_index(drop=True).groupby(
    ['dataset', 'lang']
)[['accuracy', 'macro_f1', 'weighted_f1']].mean()

accuracy  macro_f1  weighted_f1
dataset         lang                                 
tweetsentimulti arb   0.496223  0.495502     0.495502
                deu   0.508374  0.495648     0.495648
                fra   0.535140  0.531229     0.531229
                hin   0.419704  0.405560     0.405560
                ita   0.490640  0.476486     0.476486
                por   0.571921  0.557579     0.557579
                spa   0.559113  0.548840     0.548840

In [21]:
df.loc[
    (df['icl_idx'] == 'sbert') & (df['num_icl'] == 3) & 
    (df['dataset'] == 'tweetsentimulti') & (df['use_ioa'] == 'True')
,:].sort_values(['dataset', 'icl_type', 'iia_type'], ascending=False).reset_index(drop=True).groupby(
    ['dataset', 'lang']
)[['accuracy', 'macro_f1', 'weighted_f1']].mean()

accuracy  macro_f1  weighted_f1
dataset         lang                                 
tweetsentimulti arb   0.368774  0.330634     0.330634
                deu   0.337739  0.244563     0.244563
                fra   0.376245  0.313043     0.313043
                hin   0.362261  0.318814     0.318814
                ita   0.355747  0.283060     0.283060
                por   0.480651  0.439994     0.439994
                spa   0.395977  0.301777     0.301777

In [22]:
df.loc[
    (df['icl_idx'] == 'sbert') & (df['num_icl'] == 3) & 
    (df['dataset'] == 'tweetsentimulti') & (df['use_ioa'] == 'Target')
,:].sort_values(['dataset', 'icl_type', 'iia_type'], ascending=False).reset_index(drop=True).groupby(
    ['dataset', 'lang']
)[['accuracy', 'macro_f1', 'weighted_f1']].mean()

accuracy  macro_f1  weighted_f1
dataset         lang                                 
tweetsentimulti arb   0.463793  0.449383     0.449383
                deu   0.489224  0.476863     0.476863
                fra   0.486638  0.480410     0.480410
                hin   0.381753  0.362209     0.362209
                ita   0.482184  0.475893     0.475893
                por   0.557759  0.550303     0.550303
                spa   0.520115  0.505569     0.505569

# ITC

In [13]:
dfs = []
for path in glob.glob('../metrics_itc/*.csv'):
    meta = path.split('/')[-1][:-4].split('_')
    dset_name, model_name = meta[1], meta[2]
    
    meta[3] = meta[3].replace('mono-tran', 'monotrans')
    itc_meta = meta[3][4:].split('-')
    itc_num = int(itc_meta[2])
    itc_label = itc_meta[-1]
    itc_type, itc_idx = itc_meta[:2]
    
    # Load Metrics
    df = pd.read_csv(path)
    df = df.T
    df.columns = df.head(1).values[0]
    df = df.iloc[1:]

    # Inject Metadata
    df['model_name'] = model_name
    df['itc_num'] = itc_num
    df['itc_label'] = itc_label
    df['itc_type'] = itc_type
    df['itc_idx'] = itc_idx
    
    df['accuracy'] = pd.to_numeric(df['accuracy'])
    df['macro_f1'] = pd.to_numeric(df['macro_f1'])
    df['weighted_f1'] = pd.to_numeric(df['weighted_f1'])
    
    # Remove Unused Experiments
    df['itc_type'] = df['itc_type'].apply(lambda x: 'monotrans' if x == 'monotranss' else x)
    dfs.append(df)
    
df = pd.concat(dfs)

In [14]:
df.to_csv('itc_results.csv', index=False)

In [15]:
df.groupby(['dataset', 'lang']).size()

dataset              lang
americasnli          aym     24
                     bzd     12
                     cni     12
                     gn      24
                     hch     12
                     nah     12
                     oto     24
                     quy     24
                     shp     12
                     tar     12
americasnli-spa      aym     24
                     bzd     12
                     cni     12
                     gn      24
                     hch     12
                     nah     12
                     oto     24
                     quy     24
                     shp     12
                     tar     12
masakhanews          amh     24
                     hau     24
                     ibo     24
                     lug     24
                     pcm     24
                     sna     24
                     swa     24
                     xho     24
                     yor     24
nusatranslation      btk     24
              

In [16]:
df.groupby(['model_name', 'dataset']).size()

model_name  dataset            
bloom-7b1   americasnli             84
            americasnli-spa         84
            masakhanews            108
            nusatranslation         72
            nusatranslation-ind     72
            tweetsentimulti         84
xglm-7.5B   americasnli             84
            americasnli-spa         84
            masakhanews            108
            nusatranslation         72
            nusatranslation-ind     72
            tweetsentimulti         84
dtype: int64

In [17]:
for dset in df['dataset'].unique():
    print(dset)
    display(df.loc[
        (df['dataset'] == dset) & (df['model_name'] == 'bloom-7b1'),:
    ].sort_values(['dataset', 'itc_type', 'itc_idx', 'itc_num'], ascending=False).reset_index(drop=True).groupby(
        ['dataset', 'itc_type', 'itc_idx', 'itc_num', 'itc_label']
    )[['accuracy', 'macro_f1', 'weighted_f1']].mean())

americasnli


accuracy  macro_f1  \
dataset     itc_type  itc_idx itc_num itc_label                       
americasnli cross     sbert   3       Source     0.344250  0.274891   
                                      Target     0.361578  0.331619   
            mono      random  3       Source     0.337315  0.320202   
                                      Target     0.326889  0.311276   
                      sbert   3       Source     0.352896  0.326038   
                                      Target     0.369227  0.338882   
            monotrans random  3       Source     0.343061  0.333535   
                                      Target     0.326224  0.218637   
                      sbert   3       Source     0.365328  0.354679   
                                      Target     0.366553  0.342985   
            none      random  0       Source     0.337325  0.204720   
                                      Target     0.327888  0.220779   

                                                 weighted_f1  
dataset     itc_type  itc_idx itc_num itc_label               
americasnli cross     sbert   3       Source        0.274867  
                                      Target        0.331621  
            mono      random  3       Source        0.320164  
                                      Target        0.311308  
                      sbert   3       Source        0.325997  
                                      Target        0.338901  
            monotrans random  3       Source        0.333548  
                                      Target        0.218733  
                      sbert   3       Source        0.354689  
                                      Target        0.343018  
            none      random  0       Source        0.204766  
                                      Target        0.220885

nusatranslation-ind


accuracy  macro_f1  \
dataset             itc_type  itc_idx itc_num itc_label                       
nusatranslation-ind cross     sbert   3       Source     0.711111  0.632627   
                                              Target     0.626389  0.596224   
                    mono      random  3       Source     0.682917  0.627571   
                                              Target     0.560139  0.548612   
                              sbert   3       Source     0.681667  0.630190   
                                              Target     0.608611  0.582301   
                    monotrans random  3       Source     0.712778  0.619845   
                                              Target     0.692083  0.491876   
                              sbert   3       Source     0.634583  0.618778   
                                              Target     0.612917  0.595671   
                    none      random  0       Source     0.485833  0.483957   
                                              Target     0.684167  0.516339   

                                                         weighted_f1  
dataset             itc_type  itc_idx itc_num itc_label               
nusatranslation-ind cross     sbert   3       Source        0.702398  
                                              Target        0.640042  
                    mono      random  3       Source        0.686564  
                                              Target        0.577245  
                              sbert   3       Source        0.686806  
                                              Target        0.623699  
                    monotrans random  3       Source        0.697115  
                                              Target        0.622395  
                              sbert   3       Source        0.647206  
                                              Target        0.628047  
                    none      random  0       Source        0.480526  
                                              Target        0.632231

tweetsentimulti


...

accuracy  macro_f1  \
dataset     itc_type  itc_idx itc_num itc_label                       
masakhanews cross     sbert   3       Source     0.521032  0.469750   
                                      Target     0.309041  0.266197   
            mono      random  3       Source     0.540962  0.500634   
                                      Target     0.301635  0.275527   
                      sbert   3       Source     0.489641  0.449491   
                                      Target     0.301484  0.271869   
            monotrans random  3       Source     0.537309  0.483930   
                                      Target     0.260577  0.186201   
                      sbert   3       Source     0.510579  0.452393   
                                      Target     0.288288  0.264854   
            none      random  0       Source     0.495173  0.443690   
                                      Target     0.266966  0.197641   

                                                 weighted_f1  
dataset     itc_type  itc_idx itc_num itc_label               
masakhanews cross     sbert   3       Source        0.514745  
                                      Target        0.302322  
            mono      random  3       Source        0.541375  
                                      Target        0.298227  
                      sbert   3       Source        0.493777  
                                      Target        0.300385  
            monotrans random  3       Source        0.527658  
                                      Target        0.183690  
                      sbert   3       Source        0.498743  
                                      Target        0.291845  
            none      random  0       Source        0.469955  
                                      Target        0.190671

americasnli-spa


accuracy  macro_f1  \
dataset         itc_type  itc_idx itc_num itc_label                       
americasnli-spa cross     sbert   3       Source     0.347843  0.334958   
                                          Target     0.356898  0.310092   
                mono      random  3       Source     0.339293  0.310772   
                                          Target     0.331219  0.308905   
                          sbert   3       Source     0.351755  0.308019   
                                          Target     0.374904  0.346799   
                monotrans random  3       Source     0.339802  0.313119   
                                          Target     0.330557  0.225455   
                          sbert   3       Source     0.352802  0.317509   
                                          Target     0.367231  0.356421   
                none      random  0       Source     0.327004  0.194415   
                                          Target     0.328554  0.218837   

                                                     weighted_f1  
dataset         itc_type  itc_idx itc_num itc_label               
americasnli-spa cross     sbert   3       Source        0.334967  
                                          Target        0.310109  
                mono      random  3       Source        0.310771  
                                          Target        0.308935  
                          sbert   3       Source        0.308025  
                                          Target        0.346838  
                monotrans random  3       Source        0.313142  
                                          Target        0.225552  
                          sbert   3       Source        0.317521  
                                          Target        0.356461  
                none      random  0       Source        0.194497  
                                          Target        0.218943

nusatranslation


accuracy  macro_f1  \
dataset         itc_type  itc_idx itc_num itc_label                       
nusatranslation cross     sbert   3       Source     0.576111  0.568764   
                                          Target     0.599167  0.572804   
                mono      random  3       Source     0.703333  0.650014   
                                          Target     0.663611  0.575045   
                          sbert   3       Source     0.577639  0.572278   
                                          Target     0.682083  0.579673   
                monotrans random  3       Source     0.594306  0.584225   
                                          Target     0.692083  0.491876   
                          sbert   3       Source     0.718194  0.616683   
                                          Target     0.624167  0.588278   
                none      random  0       Source     0.563611  0.550469   
                                          Target     0.694028  0.494404   

                                                     weighted_f1  
dataset         itc_type  itc_idx itc_num itc_label               
nusatranslation cross     sbert   3       Source        0.583726  
                                          Target        0.615604  
                mono      random  3       Source        0.706141  
                                          Target        0.653810  
                          sbert   3       Source        0.584133  
                                          Target        0.664926  
                monotrans random  3       Source        0.609153  
                                          Target        0.622395  
                          sbert   3       Source        0.697866  
                                          Target        0.637178  
                none      random  0       Source        0.580521  
                                          Target        0.624453

In [10]:
for dset in df['dataset'].unique():
    print(dset)
    display(df.loc[
        (df['dataset'] == dset) & (df['model_name'] == 'xglm-7.5B'),:
    ].sort_values(['dataset', 'itc_type', 'itc_idx', 'itc_num'], ascending=False).reset_index(drop=True).groupby(
        ['dataset', 'itc_type', 'itc_idx', 'itc_num', 'itc_label']
    )[['accuracy', 'macro_f1', 'weighted_f1']].mean())

americasnli


accuracy  macro_f1  \
dataset     itc_type  itc_idx itc_num itc_label                       
americasnli cross     sbert   3       Source     0.339729  0.308738   
                                      Target     0.351223  0.322351   
            mono      random  3       Source     0.332783  0.323028   
                                      Target     0.347899  0.328278   
                      sbert   3       Source     0.341750  0.335632   
                                      Target     0.354887  0.335756   
            monotrans random  3       Source     0.321241  0.303150   
                                      Target     0.339223  0.222536   
                      sbert   3       Source     0.341961  0.331142   
                                      Target     0.357559  0.328131   
            none      random  0       Source     0.333956  0.176640   
                                      Target     0.331889  0.212176   

                                                 weighted_f1  
dataset     itc_type  itc_idx itc_num itc_label               
americasnli cross     sbert   3       Source        0.308766  
                                      Target        0.322366  
            mono      random  3       Source        0.323061  
                                      Target        0.328294  
                      sbert   3       Source        0.335652  
                                      Target        0.335774  
            monotrans random  3       Source        0.303199  
                                      Target        0.222645  
                      sbert   3       Source        0.331174  
                                      Target        0.328168  
            none      random  0       Source        0.176751  
                                      Target        0.212288

nusatranslation-ind


accuracy  macro_f1  \
dataset             itc_type  itc_idx itc_num itc_label                       
nusatranslation-ind cross     sbert   3       Source     0.542639  0.528322   
                                              Target     0.415000  0.408314   
                    mono      random  3       Source     0.593611  0.552994   
                                              Target     0.389444  0.381364   
                              sbert   3       Source     0.584167  0.536941   
                                              Target     0.507083  0.499621   
                    monotrans random  3       Source     0.510972  0.507533   
                                              Target     0.418750  0.407529   
                              sbert   3       Source     0.532222  0.524205   
                                              Target     0.522778  0.514109   
                    none      random  0       Source     0.635417  0.526079   
                                              Target     0.451944  0.445041   

                                                         weighted_f1  
dataset             itc_type  itc_idx itc_num itc_label               
nusatranslation-ind cross     sbert   3       Source        0.561886  
                                              Target        0.383759  
                    mono      random  3       Source        0.607954  
                                              Target        0.352695  
                              sbert   3       Source        0.596861  
                                              Target        0.522164  
                    monotrans random  3       Source        0.523641  
                                              Target        0.385880  
                              sbert   3       Source        0.548406  
                                              Target        0.540263  
                    none      random  0       Source        0.618864  
                                              Target        0.444980

tweetsentimulti


...

accuracy  macro_f1  \
dataset     itc_type  itc_idx itc_num itc_label                       
masakhanews cross     sbert   3       Source     0.414223  0.370677   
                                      Target     0.276631  0.229645   
            mono      random  3       Source     0.367041  0.326019   
                                      Target     0.251022  0.218314   
                      sbert   3       Source     0.360968  0.331789   
                                      Target     0.263621  0.229772   
            monotrans random  3       Source     0.395304  0.361637   
                                      Target     0.263080  0.175770   
                      sbert   3       Source     0.421627  0.378920   
                                      Target     0.279008  0.244317   
            none      random  0       Source     0.418643  0.381753   
                                      Target     0.253255  0.167735   

                                                 weighted_f1  
dataset     itc_type  itc_idx itc_num itc_label               
masakhanews cross     sbert   3       Source        0.406992  
                                      Target        0.259428  
            mono      random  3       Source        0.356988  
                                      Target        0.238769  
                      sbert   3       Source        0.362644  
                                      Target        0.249635  
            monotrans random  3       Source        0.393204  
                                      Target        0.184153  
                      sbert   3       Source        0.421225  
                                      Target        0.265113  
            none      random  0       Source        0.395161  
                                      Target        0.172457

americasnli-spa


accuracy  macro_f1  \
dataset         itc_type  itc_idx itc_num itc_label                       
americasnli-spa cross     sbert   3       Source     0.337700  0.304459   
                                          Target     0.345884  0.306762   
                mono      random  3       Source     0.340529  0.337250   
                                          Target     0.330893  0.312128   
                          sbert   3       Source     0.354273  0.337676   
                                          Target     0.355555  0.329136   
                monotrans random  3       Source     0.340769  0.320947   
                                          Target     0.339223  0.221919   
                          sbert   3       Source     0.344361  0.323967   
                                          Target     0.351230  0.320675   
                none      random  0       Source     0.332889  0.171328   
                                          Target     0.329556  0.215780   

                                                     weighted_f1  
dataset         itc_type  itc_idx itc_num itc_label               
americasnli-spa cross     sbert   3       Source        0.304484  
                                          Target        0.306780  
                mono      random  3       Source        0.337263  
                                          Target        0.312143  
                          sbert   3       Source        0.337697  
                                          Target        0.329167  
                monotrans random  3       Source        0.320980  
                                          Target        0.222029  
                          sbert   3       Source        0.324006  
                                          Target        0.320714  
                none      random  0       Source        0.171441  
                                          Target        0.215892

nusatranslation


accuracy  macro_f1  \
dataset         itc_type  itc_idx itc_num itc_label                       
nusatranslation cross     sbert   3       Source     0.547500  0.533609   
                                          Target     0.504861  0.501013   
                mono      random  3       Source     0.508611  0.505284   
                                          Target     0.395972  0.388669   
                          sbert   3       Source     0.584167  0.536941   
                                          Target     0.420833  0.416575   
                monotrans random  3       Source     0.628194  0.570368   
                                          Target     0.451111  0.443955   
                          sbert   3       Source     0.534306  0.526124   
                                          Target     0.540556  0.525284   
                none      random  0       Source     0.604306  0.530676   
                                          Target     0.417639  0.407555   

                                                     weighted_f1  
dataset         itc_type  itc_idx itc_num itc_label               
nusatranslation cross     sbert   3       Source        0.566170  
                                          Target        0.518397  
                mono      random  3       Source        0.519644  
                                          Target        0.361612  
                          sbert   3       Source        0.596861  
                                          Target        0.397701  
                monotrans random  3       Source        0.634837  
                                          Target        0.439174  
                          sbert   3       Source        0.551386  
                                          Target        0.560074  
                none      random  0       Source        0.605524  
                                          Target        0.388932

In [9]:
for dset in df['dataset'].unique():
    print(dset)
    display(df.loc[
        (df['dataset'] == dset) & (df['model_name'] == 'xglm-7.5B'),:
    ].sort_values(['dataset', 'itc_type', 'itc_idx', 'itc_num'], ascending=False).reset_index(drop=True).groupby(
        ['dataset', 'itc_type', 'itc_idx', 'itc_num', 'itc_label']
    )[['accuracy', 'macro_f1', 'weighted_f1']].mean())

americasnli


accuracy  macro_f1  \
dataset     itc_type  itc_idx itc_num itc_label                       
americasnli cross     sbert   3       Source     0.339729  0.308738   
                                      Target     0.351223  0.322351   
            mono      random  3       Source     0.332783  0.323028   
                                      Target     0.347899  0.328278   
                      sbert   3       Source     0.341750  0.335632   
                                      Target     0.354887  0.335756   
            monotrans random  3       Source     0.321241  0.303150   
                                      Target     0.339223  0.222536   
                      sbert   3       Source     0.341961  0.331142   
                                      Target     0.357559  0.328131   
            none      random  0       Source     0.333956  0.176640   
                                      Target     0.331889  0.212176   

                                                 weighted_f1  
dataset     itc_type  itc_idx itc_num itc_label               
americasnli cross     sbert   3       Source        0.308766  
                                      Target        0.322366  
            mono      random  3       Source        0.323061  
                                      Target        0.328294  
                      sbert   3       Source        0.335652  
                                      Target        0.335774  
            monotrans random  3       Source        0.303199  
                                      Target        0.222645  
                      sbert   3       Source        0.331174  
                                      Target        0.328168  
            none      random  0       Source        0.176751  
                                      Target        0.212288

nusatranslation-ind


accuracy  macro_f1  \
dataset             itc_type  itc_idx itc_num itc_label                       
nusatranslation-ind cross     sbert   3       Source     0.542639  0.528322   
                                              Target     0.415000  0.408314   
                    mono      random  3       Source     0.593611  0.552994   
                                              Target     0.389444  0.381364   
                              sbert   3       Source     0.584167  0.536941   
                                              Target     0.507083  0.499621   
                    monotrans random  3       Source     0.510972  0.507533   
                                              Target     0.418750  0.407529   
                              sbert   3       Source     0.532222  0.524205   
                                              Target     0.522778  0.514109   
                    none      random  0       Source     0.635417  0.526079   
                                              Target     0.451944  0.445041   

                                                         weighted_f1  
dataset             itc_type  itc_idx itc_num itc_label               
nusatranslation-ind cross     sbert   3       Source        0.561886  
                                              Target        0.383759  
                    mono      random  3       Source        0.607954  
                                              Target        0.352695  
                              sbert   3       Source        0.596861  
                                              Target        0.522164  
                    monotrans random  3       Source        0.523641  
                                              Target        0.385880  
                              sbert   3       Source        0.548406  
                                              Target        0.540263  
                    none      random  0       Source        0.618864  
                                              Target        0.444980

tweetsentimulti


...

accuracy  macro_f1  \
dataset     itc_type  itc_idx itc_num itc_label                       
masakhanews cross     sbert   3       Source     0.414223  0.370677   
                                      Target     0.276631  0.229645   
            mono      random  3       Source     0.367041  0.326019   
                                      Target     0.251022  0.218314   
                      sbert   3       Source     0.360968  0.331789   
                                      Target     0.263621  0.229772   
            monotrans random  3       Source     0.395304  0.361637   
                                      Target     0.263080  0.175770   
                      sbert   3       Source     0.421627  0.378920   
                                      Target     0.279008  0.244317   
            none      random  0       Source     0.418643  0.381753   
                                      Target     0.253255  0.167735   

                                                 weighted_f1  
dataset     itc_type  itc_idx itc_num itc_label               
masakhanews cross     sbert   3       Source        0.406992  
                                      Target        0.259428  
            mono      random  3       Source        0.356988  
                                      Target        0.238769  
                      sbert   3       Source        0.362644  
                                      Target        0.249635  
            monotrans random  3       Source        0.393204  
                                      Target        0.184153  
                      sbert   3       Source        0.421225  
                                      Target        0.265113  
            none      random  0       Source        0.395161  
                                      Target        0.172457

americasnli-spa


accuracy  macro_f1  \
dataset         itc_type  itc_idx itc_num itc_label                       
americasnli-spa cross     sbert   3       Source     0.337700  0.304459   
                                          Target     0.345884  0.306762   
                mono      random  3       Source     0.340529  0.337250   
                                          Target     0.330893  0.312128   
                          sbert   3       Source     0.354273  0.337676   
                                          Target     0.355555  0.329136   
                monotrans random  3       Source     0.340769  0.320947   
                                          Target     0.339223  0.221919   
                          sbert   3       Source     0.344361  0.323967   
                                          Target     0.351230  0.320675   
                none      random  0       Source     0.332889  0.171328   
                                          Target     0.329556  0.215780   

                                                     weighted_f1  
dataset         itc_type  itc_idx itc_num itc_label               
americasnli-spa cross     sbert   3       Source        0.304484  
                                          Target        0.306780  
                mono      random  3       Source        0.337263  
                                          Target        0.312143  
                          sbert   3       Source        0.337697  
                                          Target        0.329167  
                monotrans random  3       Source        0.320980  
                                          Target        0.222029  
                          sbert   3       Source        0.324006  
                                          Target        0.320714  
                none      random  0       Source        0.171441  
                                          Target        0.215892

nusatranslation


accuracy  macro_f1  \
dataset         itc_type  itc_idx itc_num itc_label                       
nusatranslation cross     sbert   3       Source     0.547500  0.533609   
                                          Target     0.504861  0.501013   
                mono      random  3       Source     0.508611  0.505284   
                                          Target     0.395972  0.388669   
                          sbert   3       Source     0.584167  0.536941   
                                          Target     0.420833  0.416575   
                monotrans random  3       Source     0.628194  0.570368   
                                          Target     0.451111  0.443955   
                          sbert   3       Source     0.534306  0.526124   
                                          Target     0.540556  0.525284   
                none      random  0       Source     0.604306  0.530676   
                                          Target     0.417639  0.407555   

                                                     weighted_f1  
dataset         itc_type  itc_idx itc_num itc_label               
nusatranslation cross     sbert   3       Source        0.566170  
                                          Target        0.518397  
                mono      random  3       Source        0.519644  
                                          Target        0.361612  
                          sbert   3       Source        0.596861  
                                          Target        0.397701  
                monotrans random  3       Source        0.634837  
                                          Target        0.439174  
                          sbert   3       Source        0.551386  
                                          Target        0.560074  
                none      random  0       Source        0.605524  
                                          Target        0.388932

In [26]:
for dset in df['dataset'].unique():
    print(dset)
    display(df.loc[
        (df['dataset'] == dset),:
    ].sort_values(['dataset', 'lang', 'itc_type', 'itc_idx', 'itc_num'], ascending=False).reset_index(drop=True).groupby(
        ['dataset', 'lang', 'itc_type', 'itc_idx', 'itc_num', 'itc_label']
    )[['accuracy', 'macro_f1', 'weighted_f1']].mean())

americasnli


accuracy  macro_f1  \
dataset     lang itc_type  itc_idx itc_num itc_label                       
americasnli aym  cross     sbert   3       Source     0.357333  0.273314   
                                           Target     0.370667  0.324494   
                 mono      random  3       Source     0.326667  0.311243   
                                           Target     0.309333  0.300666   
                           sbert   3       Source     0.345333  0.310429   
...                                                        ...       ...   
            tar  mono      random  3       Source     0.360000  0.331034   
                           sbert   3       Source     0.349333  0.306043   
                 monotrans random  3       Source     0.344000  0.327584   
                           sbert   3       Source     0.357333  0.336379   
                 none      random  0       Source     0.325333  0.188641   

                                                      weighted_f1  
dataset     lang itc_type  itc_idx itc_num itc_label               
americasnli aym  cross     sbert   3       Source        0.273314  
                                           Target        0.324494  
                 mono      random  3       Source        0.311243  
                                           Target        0.300666  
                           sbert   3       Source        0.310429  
...                                                           ...  
            tar  mono      random  3       Source        0.331034  
                           sbert   3       Source        0.306043  
                 monotrans random  3       Source        0.327584  
                           sbert   3       Source        0.336379  
                 none      random  0       Source        0.188641  

[84 rows x 3 columns]

nusatranslation-ind


accuracy  \
dataset             lang itc_type  itc_idx itc_num itc_label             
nusatranslation-ind btk  cross     sbert   3       Source     0.685833   
                                                   Target     0.585833   
                         mono      random  3       Source     0.638333   
                                                   Target     0.535833   
                                   sbert   3       Source     0.650833   
...                                                                ...   
                    sun  monotrans random  3       Target     0.688333   
                                   sbert   3       Source     0.735000   
                                                   Target     0.659167   
                         none      random  0       Source     0.533333   
                                                   Target     0.678333   

                                                              macro_f1  \
dataset             lang itc_type  itc_idx itc_num itc_label             
nusatranslation-ind btk  cross     sbert   3       Source     0.611465   
                                                   Target     0.565268   
                         mono      random  3       Source     0.583328   
                                                   Target     0.526441   
                                   sbert   3       Source     0.600213   
...                                                                ...   
                    sun  monotrans random  3       Target     0.520462   
                                   sbert   3       Source     0.706826   
                                                   Target     0.641133   
                         none      random  0       Source     0.532315   
                                                   Target     0.552682   

                                                              weighted_f1  
dataset             lang itc_type  itc_idx itc_num itc_label               
nusatranslation-ind btk  cross     sbert   3       Source        0.681442  
                                                   Target        0.604193  
                         mono      random  3       Source        0.645651  
                                                   Target        0.553896  
                                   sbert   3       Source        0.658776  
...                                                                   ...  
                    sun  monotrans random  3       Target        0.637262  
                                   sbert   3       Source        0.744240  
                                                   Target        0.674250  
                         none      random  0       Source        0.541300  
                                                   Target        0.650279  

[72 rows x 3 columns]

americasnli-spa


...

accuracy  macro_f1  \
dataset         lang itc_type  itc_idx itc_num itc_label                       
nusatranslation btk  cross     sbert   3       Source     0.508333  0.507478   
                                               Target     0.544167  0.525008   
                     mono      random  3       Source     0.683333  0.627665   
                                               Target     0.646667  0.551835   
                               sbert   3       Source     0.507500  0.507500   
...                                                            ...       ...   
                sun  monotrans random  3       Target     0.688333  0.520462   
                               sbert   3       Source     0.754167  0.654288   
                                               Target     0.672500  0.641089   
                     none      random  0       Source     0.610000  0.590790   
                                               Target     0.690833  0.520659   

                                                          weighted_f1  
dataset         lang itc_type  itc_idx itc_num itc_label               
nusatranslation btk  cross     sbert   3       Source        0.515926  
                                               Target        0.564279  
                     mono      random  3       Source        0.686933  
                                               Target        0.636702  
                               sbert   3       Source        0.507669  
...                                                               ...  
                sun  monotrans random  3       Target        0.637262  
                               sbert   3       Source        0.730784  
                                               Target        0.684799  
                     none      random  0       Source        0.627289  
                                               Target        0.638234  

[72 rows x 3 columns]

masakhanews


accuracy  macro_f1  \
dataset     lang itc_type  itc_idx itc_num itc_label                       
masakhanews amh  cross     sbert   3       Source     0.340426  0.282941   
                                           Target     0.313830  0.309301   
                 mono      random  3       Source     0.257979  0.230096   
                                           Target     0.297872  0.290406   
                           sbert   3       Source     0.265957  0.237487   
...                                                        ...       ...   
            yor  monotrans random  3       Target     0.177616  0.110499   
                           sbert   3       Source     0.632603  0.605428   
                                           Target     0.223844  0.214030   
                 none      random  0       Source     0.661800  0.670669   
                                           Target     0.177616  0.113200   

                                                      weighted_f1  
dataset     lang itc_type  itc_idx itc_num itc_label               
masakhanews amh  cross     sbert   3       Source        0.284057  
                                           Target        0.309549  
                 mono      random  3       Source        0.229896  
                                           Target        0.288645  
                           sbert   3       Source        0.241671  
...                                                           ...  
            yor  monotrans random  3       Target        0.096069  
                           sbert   3       Source        0.617700  
                                           Target        0.216587  
                 none      random  0       Source        0.661422  
                                           Target        0.097937  

[108 rows x 3 columns]

tweetsentimulti


accuracy  macro_f1  \
dataset         lang itc_type  itc_idx itc_num itc_label                       
tweetsentimulti arb  cross     sbert   3       Source     0.513793  0.514501   
                                               Target     0.488506  0.440902   
                     mono      random  3       Source     0.414943  0.406649   
                                               Target     0.350575  0.321950   
                               sbert   3       Source     0.516092  0.499507   
...                                                            ...       ...   
                spa  monotrans random  3       Target     0.465517  0.466526   
                               sbert   3       Source     0.578161  0.566051   
                                               Target     0.568966  0.568269   
                     none      random  0       Source     0.471264  0.434950   
                                               Target     0.465517  0.465953   

                                                          weighted_f1  
dataset         lang itc_type  itc_idx itc_num itc_label               
tweetsentimulti arb  cross     sbert   3       Source        0.514501  
                                               Target        0.440902  
                     mono      random  3       Source        0.406649  
                                               Target        0.321950  
                               sbert   3       Source        0.499507  
...                                                               ...  
                spa  monotrans random  3       Target        0.466526  
                               sbert   3       Source        0.566051  
                                               Target        0.568269  
                     none      random  0       Source        0.434950  
                                               Target        0.465953  

[84 rows x 3 columns]